# CNN Model for Driver Drowsiness Detection

### CNN Model

### Regularization, Batch Normalization pending

## Import libraries

In [19]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import lite

## Data Preprocessing

    ### Preprocessing the training & testing dataset

In [20]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range=[0.2,1.0],
    height_shift_range=0.5,
    rotation_range=90,
    brightness_range=[0.2,1.0],
    
    horizontal_flip = True
)

train_set = train_datagen.flow_from_directory(
    './dataset_new/train/eyes',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_set = test_datagen.flow_from_directory(
    './dataset_new/test/eyes',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [21]:
print(train_set.class_indices)

{'Closed': 0, 'Open': 1}


## Building the CNN

### Initializing the CNN

In [22]:
cnn = tf.keras.models.Sequential()

### Convolution Operation

In [23]:
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size=3,activation = 'relu', input_shape = [64,64,3]))

### Pooling

In [24]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolution layer

In [25]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))    
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Flattening

In [26]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [27]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Output Layer

In [28]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### Training the CNN

In [29]:
cnn.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

Epoch 1/25
39/39 [==============================] - 11s 249ms/step - loss: 0.6710 - accuracy: 0.6062 - val_loss: 0.7008 - val_accuracy: 0.5000
Epoch 2/25
39/39 [==============================] - 7s 186ms/step - loss: 0.6163 - accuracy: 0.6645 - val_loss: 0.7986 - val_accuracy: 0.5138
Epoch 3/25
39/39 [==============================] - 8s 201ms/step - loss: 0.5700 - accuracy: 0.7099 - val_loss: 0.8134 - val_accuracy: 0.6009
Epoch 4/25
39/39 [==============================] - 8s 204ms/step - loss: 0.5256 - accuracy: 0.7318 - val_loss: 0.5004 - val_accuracy: 0.8119
Epoch 5/25
39/39 [==============================] - 8s 216ms/step - loss: 0.5505 - accuracy: 0.7245 - val_loss: 0.6439 - val_accuracy: 0.6927
Epoch 6/25
39/39 [==============================] - 11s 279ms/step - loss: 0.5101 - accuracy: 0.7553 - val_loss: 0.7693 - val_accuracy: 0.6422
Epoch 7/25
39/39 [==============================] - 9s 240ms/step - loss: 0.5132 - accuracy: 0.7488 - val_loss: 0.6548 - val_accuracy: 0.7110
Epoc

In [30]:
test_labels = ['Closed','Open']
test_loss, test_acc = cnn.evaluate(test_set, verbose=2)

7/7 - 1s - loss: 0.4025 - accuracy: 0.8394 - 549ms/epoch - 78ms/step


In [31]:
print(test_acc)
print(test_loss)

0.8394495248794556
0.4025290310382843


### Saving CNN model

In [32]:
# serialize model to JSON
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tfmodel = converter.convert()
open("linear.tflite","wb").write(tfmodel)

print("Saved model to disk")

INFO:tensorflow:Assets written to: C:\Users\JATIND~1\AppData\Local\Temp\tmpht0iu73m\assets


INFO:tensorflow:Assets written to: C:\Users\JATIND~1\AppData\Local\Temp\tmpht0iu73m\assets


Saved model to disk


In [33]:
path = "./dataset_new/train/"